In [ ]:
import torch
import random

from typing import Dict, List

from allennlp.data.fields import Field, TextField, ListField, MetadataField, IndexField,ArrayField
from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer
from allennlp.data.tokenizers import Tokenizer, WordTokenizer
from torch.nn.functional import nll_loss, binary_cross_entropy

from allennlp.modules.token_embedders import Embedding
from allennlp.data.iterators import BucketIterator, BasicIterator
from allennlp.modules.text_field_embedders import TextFieldEmbedder, BasicTextFieldEmbedder
from allennlp.models.model import Model
from allennlp.data import Vocabulary
from allennlp.modules import Seq2SeqEncoder, SimilarityFunction, TimeDistributed, TextFieldEmbedder,Seq2VecEncoder, MatrixAttention
from allennlp.nn import util, InitializerApplicator, RegularizerApplicator
from typing import Any, Dict, List, Optional

from allennlp.modules import InputVariationalDropout
from allennlp.modules.seq2seq_encoders import Seq2SeqEncoder, PytorchSeq2SeqWrapper, StackedBidirectionalLstm
from allennlp.modules.matrix_attention import LinearMatrixAttention
from allennlp.training.metrics import BooleanAccuracy, Auc

from self_attentive import SelfAttentive
from qangaroo import MyQangarooReader
from decoder import Decoder, SANDecoder
from allennlp.data.iterators.data_iterator import DataIterator, TensorDict
from model import MultiStepParaRankModel

%load_ext autoreload
%autoreload 2

In [ ]:
from allennlp.common import Params
from allennlp.data.dataset_readers import DatasetReader
from allennlp.training.util import create_serialization_dir
from allennlp.training.trainer import Trainer, TrainerPieces
from allennlp.commands import train

In [ ]:
param_path = './model.jsonnet'
serialization_dir = 'checkpoints_test'
#train.train_model_from_file(param_path, serialization_dir, force=True)

In [ ]:
dev_json = '../data/qangaroo_v1.1/wikihop/dev.json'
toy_data = './toy_data.json'


### Debug Train

In [ ]:
serialization_dir = './temp_dir_1'
params = Params.from_file(param_path)
create_serialization_dir(params, serialization_dir, False,False)
pieces = TrainerPieces.from_params(params, serialization_dir)  # pylint: disable=no-member


In [ ]:
trainer = Trainer.from_params(
            model=pieces.model,
            serialization_dir=serialization_dir,
            iterator=pieces.iterator,
            train_data=pieces.train_dataset,
            validation_data=pieces.validation_dataset,
            params=pieces.params,
            validation_iterator=pieces.validation_iterator)   

In [ ]:
from allennlp.common.util import lazy_groups_of
import math
from allennlp.common.tqdm import Tqdm

In [ ]:
from allennlp.data import Vocabulary
from allennlp.data import iterators

iterator = iterators.BasicIterator(batch_size=4)
iterator.index_with(trainer.model.vocab)
#raw_iterator = iterator(trainer.train_data)

In [ ]:
raw_train_generator = iterator(trainer.train_data,
                                            num_epochs=2,
                                            shuffle=trainer.shuffle)
    
train_generator = lazy_groups_of(raw_train_generator, 1)
num_training_batches = math.ceil(trainer.iterator.get_num_batches(trainer.train_data)/1)    
train_generator_tqdm = Tqdm.tqdm(train_generator, total=num_training_batches)

In [ ]:
for batch_group in train_generator_tqdm:
    trainer.optimizer.zero_grad()

    loss = trainer.batch_loss(batch_group, for_training=True)

    if torch.isnan(loss):
        raise ValueError("nan loss encountered")

    loss.backward()

In [ ]:
with torch.autograd.profiler.profile(use_cuda=True) as prof:
    trainer.optimizer.zero_grad()

    loss = trainer.batch_loss(batch_group, for_training=True)

    if torch.isnan(loss):
        raise ValueError("nan loss encountered")

    #loss.backward()

In [ ]:
print(prof.key_averages())

### Debug Model

In [ ]:
import pixiedust

In [ ]:
phrase_layer = PytorchSeq2SeqWrapper(StackedBidirectionalLstm(300,100,1,0.2, 0.2,True))
pq_attention = LinearMatrixAttention(200,200,'x,y,x*y')
p_selfattention = LinearMatrixAttention(400,400,'x,y,x*y')

supports_pooling = SelfAttentive(400)
question_pooling = SelfAttentive(200)
candidates_pooling = SelfAttentive(200)

decoder = SANDecoder(support_dim=400,candidates_dim=200, query_dim=200, num_step=3, 
                     dropout_p=0.2, reason_dropout_p=0.4, reason_type=1)

model = MultiStepParaRankModel(vocab, word_embeddings, phrase_layer,
                               pq_attention, p_selfattention, 
                               supports_pooling, question_pooling, candidates_pooling,decoder)

In [ ]:
%%pixie_debugger

output = model(**batch)